In [1]:
import os
import sys
from utils.common import load_object,read_yaml,create_directories
from exception.exception import customexception
from logger.logger import logging
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
#config_entity
@dataclass(frozen=True)
class ModelPredictionConfig:
    preprocessor_obj_path: Path
    model_path: Path


In [3]:
from constants import CONFIG_FILE_PATH
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prediction_config(self)->ModelPredictionConfig:
        training_config = self.config.model_training
        datatransform_config = self.config.data_transformation
        
        preprocessor_path = os.path.join(datatransform_config.preprocessed_data_files,"preprocessor.pkl")
        
        prediction_config = ModelPredictionConfig(
            preprocessor_obj_path=preprocessor_path,
            model_path=training_config.model_path
        )
        
        return prediction_config

In [4]:
# component file
class ModelPrediction:
    def __init__(self,config:ModelPredictionConfig):
        self.config = config
    
    def predict(self,input_data):
        try:
            preprocessor = load_object(self.config.preprocessor_obj_path)
            model = load_object(self.config.model_path)
            
            transform_data = preprocessor.transform(input_data)
            print(transform_data)
            
            prediction_result = model.predict(transform_data)
            print(prediction_result)
            
            logging.info(f"Prediction result : {prediction_result}")
            return prediction_result
            
        except Exception as e:
            raise customexception(e,sys)

In [5]:
cd ..

e:\Programming\Projects\Machine Learning Projects\GemStonePricePrediction


In [7]:

try:
    logging.info("Model prediction started")
    config = ConfigurationManager()
    model_prediction_Config = config.get_prediction_config()
    model_prediction = ModelPrediction(model_prediction_Config)
    input_dict = {
                'carat':0.7,
                'depth':63.0,
                'table':56.0,
                'x':5.66,
                'y':5.69,
                'z':3.58,
                'cut':'Very Good',
                'color':'G',
                'clarity':'VS2'
                }
    input_data = pd.DataFrame([input_dict])
    model_prediction.predict(input_data)
    logging.info("Model prediction completed")
except Exception as e:
    raise customexception(e,sys)

[[-0.1944146   1.09005228 -0.63937207 -0.04821015 -0.02557611  0.06828201
   2.          3.          3.        ]]
[2512.7354]
